In [1]:
import sys
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/')

In [2]:
import torch
import pandas as pd
import numpy as np
import statistics
import torch
import random
import time
import numpy as np
#from transformers import AdamW
from torch.utils.tensorboard import SummaryWriter
import pickle
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import shutil
import itertools
from model.GRU_model import *
from pipeline_config import *
from utils import *

In [3]:
# Seeds for comparisons:

torch.manual_seed(1)
np.random.seed(2)
random.seed(3)

In [4]:
with open(train_dataset_path, 'rb') as f:
    train_dataset_dict = pickle.load(f)
    
with open(test_dataset_path, 'rb') as f:
    test_dataset_dict = pickle.load(f)

with open(category_id_to_category_name_path, 'rb') as f:
    category_id_to_name_dict = pickle.load(f)
    
with open(categories_per_indent_path, 'rb') as f:
    categories_per_indent_dict = pickle.load(f)

In [5]:
categories_per_indent_path

'/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/pickle_files/norway_categories_per_indent_dict.pickle'

In [6]:
categories_lists = list(categories_per_indent_dict.values())
categories_id = list(itertools.chain.from_iterable(categories_lists))
categories = []
for category_id in categories_id:
    categories.append(category_id_to_name_dict[category_id])

In [7]:
# sanity check:
len(categories)

52

# Pipeline:

In [8]:
print(f'Features: {Features}')
print(f'HiddenSize: {HiddenSize}')
print(f'LayersDim: {LayersDim}')
print(f'OutputDim: {OutputDim}')
print(f'DropoutProb: {DropoutProb}')
print(f'Lr : {Lr}')
print(f'Epochs : {Epochs}')

Features: 1
HiddenSize: 64
LayersDim: 1
OutputDim: 1
DropoutProb: 0.0
Lr : 0.00019837551214953915
Epochs : 100


In [9]:
len(categories)

52

In [10]:
def pipline(train_dataset_dict, test_dataset_dict, weights_path):
    results = {}

    categories_counter = 0

    for indent in sorted(list(categories_per_indent_dict.keys())):
        for category in categories_per_indent_dict[indent]:
            print(f'indent: {indent}')
            category_name = category_id_to_name_dict[category]
            print(f'category_name: {category_name}')

            categories_counter+= 1
            print(f'categories_counter: {categories_counter}')
            
            train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category_name], test_dataset_dict[category_name])
            model = Model
            model.to(device)
            optimizer = Optimizer
            
            saving_param_path = weights_path+category_name+'.pt'

            results[category_name] = training_and_evaluation(
                                model=model,
                                train_dataloader=train_dataloader,
                                test_dataloader=test_dataloader,
                                optim=optimizer,
                                category=category,
                                path=saving_param_path
                            )

    return results

In [11]:
results = pipline(train_dataset_dict, test_dataset_dict, weightspath)

indent: 0
category_name: All-items
categories_counter: 1
indent: 1
category_name: Alcoholic beverages and tobacco
categories_counter: 2
indent: 1
category_name: Clothing and footwear
categories_counter: 3
indent: 1
category_name: Communications
categories_counter: 4
indent: 1
category_name: Education
categories_counter: 5
indent: 1
category_name: Food and non-alcoholic beverages
categories_counter: 6
indent: 1
category_name: Furnishings, household equipment and routine maintenance
categories_counter: 7
indent: 1
category_name: Health
categories_counter: 8
indent: 1
category_name: Housing, water, electricity, gas and other fuels
categories_counter: 9
indent: 1
category_name: Miscellaneous goods and services
categories_counter: 10
indent: 1
category_name: Recreation and culture
categories_counter: 11
indent: 1
category_name: Restaurants and hotels
categories_counter: 12
indent: 1
category_name: Transport
categories_counter: 13
indent: 2
category_name: Accommodation services
categories_co

Save best predictions per category:

In [12]:
categories_lists = list(categories_per_indent_dict.values())
categories_id = list(itertools.chain.from_iterable(categories_lists))
categories = []
for category_id in categories_id:
    categories.append(category_id_to_name_dict[category_id])

predictions_dict = get_results_on_test_set(weightspath, train_dataset_dict, test_dataset_dict, categories = categories)

Accommodation services
Actual rentals for housing
Alcoholic beverages
Audio-visual, photographic and information processing equipment
Clothing
Electricity, gas and other fuels
Financial services n.e.c.
Food
Footwear
Furniture and furnishings, carpets and other floor coverings
Glassware, tableware and household utensils
Goods and services for routine household maintenance
Household appliances
Household textiles
Imputed rentals for housing
Insurance
Maintenance and repair of the dwelling
Medical products, appliances and equipment
Newspapers, books and stationery
Non-alcoholic beverages
Operation of personal transport equipment
Other major durables for recreation and culture
Other recreational items and equipment, gardens and pets
Other services n.e.c.
Out-patient services
Package holidays
Personal care
Personal effects n.e.c.
Postal services
Purchase of vehicles
Recreational and cultural services
Restaurant services
Social protection
Telephone equipment
Telephone services
Tobacco
Tools a

In [13]:
with open(test_predictions_path, 'wb') as handle:
    pickle.dump(predictions_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

Save model weights in dict for bi-directional use:

In [14]:
categories_lists = list(categories_per_indent_dict.values())
categories_id = list(itertools.chain.from_iterable(categories_lists))
categories = []
for category_id in categories_id:
    categories.append(category_id_to_name_dict[category_id])

category_id_list = []
 
# list out keys and values separately
key_list = list(category_id_to_name_dict.keys())
val_list = list(category_id_to_name_dict.values())

for cat_name in categories:
    position = val_list.index(cat_name)
    category_id_list.append(key_list[position])

In [15]:
weightspath

'/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/basic/models_weights/'

In [16]:
weights_dict = get_weights_per_category(category_id_list, weightspath, category_id_to_name_dict)

In [17]:
with open(sgru_model_weights_path, 'wb') as handle:
    pickle.dump(weights_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)